In [1]:
import os
import glob
from tqdm.auto import tqdm
import shutil
import zipfile
import patoolib

import numpy as np

import utils

In [2]:
models_path = './best_models'
files_path = './processed_data'


os.makedirs(models_path, exist_ok=True)
os.makedirs(files_path, exist_ok=True)

In [3]:
class CFG:
    V = 'FINAL_MMD_JT_HL_CM'
    Vh = 'FINAL_MMD_HL'
    Vj = 'FINAL_MMD_JT'
    Vc = 'FINAL_MMD_CM'

    sub_no = 1
    note="Data Preprocessing and modelling for JTEXT, CMOD & HL_2A"

### PREPROCESS TRAINING & TESTING DATA

In [4]:
with zipfile.ZipFile("../C-Mod_data.zip", 'r') as zip_ref:
    zip_ref.extractall('../')

with zipfile.ZipFile('../HL-2A_data.zip', 'r') as zip_ref:
    zip_ref.extractall('../')

with zipfile.ZipFile('../J-TEXT_data.zip', 'r') as zip_ref:
    zip_ref.extractall('../')

with zipfile.ZipFile('../J-TEXT data/data.zip', 'r') as zip_ref:
    zip_ref.extractall('../J-TEXT data/')
    
patoolib.extract_archive('../HL-2A data/HL-2A_Data.rar', outdir='../HL-2A data/')

patool: Extracting ../HL-2A data/HL-2A_Data.rar ...
patool: running "C:\Program Files\WinRAR\rar.EXE" x -- "c:\Users\ISAAC.E.SIAW-DARKO\Desktop\Competition Projects\Multi-Machine Disruption Prediction Challenge for Fusion Energy by ITU\submission_files\best_submission\HL-2A data\HL-2A_Data.rar"
patool:     with cwd="../HL-2A data/", creationflags=134217728
patool: ... ../HL-2A data/HL-2A_Data.rar extracted to `../HL-2A data/'.


'../HL-2A data/'

In [5]:
source_jtext = "../jtext_train_repo"
source_cmod = "../training_repo"
source_hl_2a = "../hl_2a_train_repo"

test_source = "../testing_repo"

In [6]:
#1. set up the empty dirs

os.makedirs(source_cmod , exist_ok=True)
os.makedirs(test_source , exist_ok=True)
os.makedirs(source_jtext , exist_ok=True)
os.makedirs(source_hl_2a , exist_ok=True)

In [7]:
#2.use glob get all the paths to all shots

c_mod_train_paths = glob.glob('../cmod/CMod_train/*/*')
jtext_paths = glob.glob('../J-TEXT data/processed_data_1k_5k_final/*/*')
hl_2a_paths = glob.glob('../HL-2A data/JDDB_repo_2A_5k/*')

c_mod_test_paths = glob.glob('../cmod/CMod_evaluate_20_10_2023/*/*')

In [8]:
# ##3. backlash works weirdly in python so inorder to use it within our loop function we put in a variable. and set our shutil.move loop
c = "\\"
[shutil.move(path , f"{source_cmod}/{path.split(c)[-1]}") for path in tqdm(c_mod_train_paths)]

[shutil.move(path , f"{test_source}/{path.split(c)[-1]}") for path in tqdm(c_mod_test_paths)]

[shutil.move(path , f"{source_jtext}/{path.split(c)[-1]}" ) for path in tqdm(jtext_paths)]

[shutil.move(path , f"{source_hl_2a}/{path.split(c)[-1]}"  ) for path in tqdm(hl_2a_paths)]

pass

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/413 [00:00<?, ?it/s]

  0%|          | 0/2136 [00:00<?, ?it/s]

  0%|          | 0/975 [00:00<?, ?it/s]

In [9]:
jtext_repo = utils.open_repo(source_jtext)
cmod_repo = utils.open_repo(source_cmod)
hl_2a_repo = utils.open_repo(source_hl_2a)

jtext_shot_list = jtext_repo.get_all_shots()
cmod_shot_list = cmod_repo.get_all_shots()
hl_2a_shot_list = hl_2a_repo.get_all_shots()

test_repo = utils.open_repo(test_source)
test_shot_list = test_repo.get_all_shots()

In [10]:
train_x_cmod_df , train_y_cmod_df = utils.rebuild_train_files(cmod_repo , cmod_shot_list)
train_x_hl_2a_df , train_y_hl_2a_df = utils.rebuild_train_files(hl_2a_repo , hl_2a_shot_list)
train_x_jtext_df , train_y_jtext_df = utils.rebuild_train_files(jtext_repo , jtext_shot_list)

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/975 [00:00<?, ?it/s]

c:\Users\ISAAC.E.SIAW-DARKO\Desktop\Competition Projects\Multi-Machine Disruption Prediction Challenge for Fusion Energy by ITU\submission_files\best_submission\WolfPack\utils.py:73: RuntimeWarning: overflow encountered in square
  train_df_dict = {i : [np.square(df[v][i]).sum() for v in range(df.shape[0])] for i in range(df.shape[1])}
c:\Users\ISAAC.E.SIAW-DARKO\Desktop\Competition Projects\Multi-Machine Disruption Prediction Challenge for Fusion Energy by ITU\submission_files\best_submission\WolfPack\utils.py:73: RuntimeWarning: overflow encountered in square
  train_df_dict = {i : [np.square(df[v][i]).sum() for v in range(df.shape[0])] for i in range(df.shape[1])}
c:\Users\ISAAC.E.SIAW-DARKO\Desktop\Competition Projects\Multi-Machine Disruption Prediction Challenge for Fusion Energy by ITU\submission_files\best_submission\WolfPack\utils.py:73: RuntimeWarning: overflow encountered in square
  train_df_dict = {i : [np.square(df[v][i]).sum() for v in range(df.shape[0])] for i in range(

  0%|          | 0/2136 [00:00<?, ?it/s]

In [11]:
test_x_cmod_df = utils.rebuild_test_files(test_repo , test_shot_list)

  0%|          | 0/413 [00:00<?, ?it/s]

In [12]:
train_x_jtext_df.shape, train_x_hl_2a_df.shape, train_x_cmod_df.shape, test_x_cmod_df.shape

((2136, 90), (975, 86), (20, 173), (413, 198))

In [13]:
train_x_jtext_df.head(2)

,AXUV_CA_02,AXUV_CA_06,AXUV_CA_10,AXUV_CA_14,AXUV_CB_18,AXUV_CB_22,AXUV_CB_26,AXUV_CB_30,AXUV_CE_66,AXUV_CE_70,...,sxr_cc_054,sxr_cc_055,sxr_cc_056,sxr_cc_057,sxr_cc_058,sxr_cc_059,sxr_cc_060,vl,vs_c3_aa018,Shot_list
0,183.357883,154.619660,264.436769,34.200124,111.495991,95.853351,105.852292,2.509004,64.391372,46.066482,...,32.249536,20.939290,12.044595,6.843564,3.927831,2.247210,1.118488,1538.897392,82.681080,ID_1051461
1,0.004404,0.006311,0.007499,0.015826,0.001124,0.008660,0.017694,0.013897,0.005216,0.003189,...,75.704091,53.885599,33.088334,18.919968,11.068707,6.267908,3.297584,1676.810338,39.770722,ID_1051499


In [14]:
train_x_hl_2a_df.head(2)

,CCO-DF:DENSITY1,CCO-DF:GASFBOUT,CCO-LFB:LFBBOH,CCO-LFB:LFBBT,CCO-LFB:LFBBV,CCO-LFB:LFBIRF,CCO-LFB:LFBMP1,CCO-LFB:LFBMP2,CCO-LFB:LFDH,CCO-LFB:LFDV,...,EFIT_q95,Ip_Error,RadiatedFraction,RotatingModeProxy,Shot_list,DS-FIR:FIR-SPH1A,DS-FIR:FIR-SPH2A,DS-FIR:FIR-SPH3A,DS-FIR:FIR-SPH4A,GW_Ratio
0,63.195060,0.0,340196.753949,751269.061201,175834.331855,0.032328,0.299856,1.468048,1243.871699,19891.112859,...,28661.196061,28402.208371,2.529730e+08,0.035446,ID_35956,NaN,NaN,NaN,NaN,NaN
1,64.531645,0.0,311237.954988,754187.159239,171480.249900,0.041957,0.254372,0.675082,1446.225730,20278.368004,...,30456.294527,30957.379469,1.170411e+09,0.028053,ID_35957,NaN,NaN,NaN,NaN,NaN


In [15]:
train_x_cmod_df.head(2)

,.tci.results:nl_04,Greenwald_fraction,\AXA.chord_1,\AXA.chord_10,\AXA.chord_11,\AXA.chord_12,\AXA.chord_13,\AXA.chord_14,\AXA.chord_15,\AXA.chord_16,...,\xtomo::top.brightnesses.array_3:chord_35,\xtomo::top.brightnesses.array_3:chord_36,\xtomo::top.brightnesses.array_3:chord_37,\xtomo::top.brightnesses.array_3:chord_38,ip_err_frac,n1_norm,radiated_frac,rotating_mode,time_until_disrupt,Shot_list
0,5.326269e+42,65.373447,1.912998e+11,1.135554e+14,1.567546e+14,2.091652e+14,2.726298e+14,3.262958e+14,3.490044e+14,3.690343e+14,...,4.526109e+07,4.610338e+07,2.937596e+07,2.845743e+06,7.470543,0.001752,1177.011085,114.241099,0.0,ID_1120104019
1,2.616991e+38,0.002250,1.223206e+11,3.294500e+13,4.329573e+13,5.542801e+13,6.992380e+13,8.104123e+13,8.971919e+13,1.045243e+14,...,3.499170e+07,2.434394e+07,2.797387e+07,1.986765e+06,7.325798,0.002196,792.232679,118.411473,0.0,ID_1120117013


CMOD RENAMING

In [16]:
Diagnostics = "plasma current,toroidal magnetic field,horizontal displacement,vertical displacement,line integral density(center chord),C3 radiation,loop voltage,AXUV_01,AXUV_02,AXUV_03,AXUV_04,AXUV_05,AXUV_06,AXUV_07,AXUV_08,AXUV_09,AXUV_10,AXUV_11,AXUV_12,AXUV_13,AXUV_14,AXUV_15,AXUV_16,AXUV_17,AXUV_18,AXUV_19,AXUV_20,AXUV_21,AXUV_22,soft-X-ray_01,soft-X-ray_02,soft-X-ray_03,soft-X-ray_04,soft-X-ray_05,soft-X-ray_06,soft-X-ray_07,soft-X-ray_08,soft-X-ray_09,soft-X-ray_10,soft-X-ray_11,soft-X-ray_12,soft-X-ray_13,soft-X-ray_14,soft-X-ray_15,soft-X-ray_16,soft-X-ray_17,soft-X-ray_18,soft-X-ray_19,soft-X-ray_20,soft-X-ray_21,soft-X-ray_22,soft-X-ray_23,soft-X-ray_24,soft-X-ray_25,soft-X-ray_26,soft-X-ray_27,soft-X-ray_28,soft-X-ray_29,soft-X-ray_30,soft-X-ray_31,soft-X-ray_32,soft-X-ray_33,soft-X-ray_34,soft-X-ray_35,soft-X-ray_36,soft-X-ray_37,soft-X-ray_38,poloidal Mirnov probes_01,poloidal Mirnov probes_02,poloidal Mirnov probes_03,poloidal Mirnov probes_04,poloidal Mirnov probes_05,poloidal Mirnov probes_06,poloidal Mirnov probes_07,poloidal Mirnov probes_08,poloidal Mirnov probes_09,poloidal Mirnov probes_10,poloidal Mirnov probes_11,poloidal Mirnov probes_12,poloidal Mirnov probes_13,poloidal Mirnov probes_14,poloidal Mirnov probes_15,poloidal Mirnov probes_16,poloidal Mirnov probes_17,poloidal Mirnov probes_18,toroidal Mirnov probes_01,toroidal Mirnov probes_02,toroidal Mirnov probes_03,toroidal Mirnov probes_04,toroidal Mirnov probes_05,toroidal Mirnov probes_06,toroidal Mirnov probes_07,toroidal Mirnov probes_09,toroidal Mirnov probes_10,Density_Measurement1,Density_Measurement2,Density_Measurement3,Density_Measurement4,Greenwald fraction,LM proxy,q95 proxy,IP error fraction,Radiation fraction,Rotating mode proxy,unknown_BP_KA_TOP,unknown_BP_EF_TOP,unknown_BP_KA_BOT,unknown_aminor,unknown_betan,unknown_kappa,unknown_li,unknown_wplasm".replace(' ','_').replace('-', '_').lower().split(',')
cmod_feats =["\\ip", "\\btor", '\\efit_aeqdsk:rmagx', '\\efit_aeqdsk:zmagx', '.tci.results:nl_04','\\twopi_diode', '\\efit_aeqdsk:vloopt','\\AXA.chord_1 & \\AXJ.chord_1', '\\AXA.chord_2 & \\AXJ.chord_2', '\\AXA.chord_3 & \\AXJ.chord_3', '\\AXA.chord_4 & \\AXJ.chord_4', '\\AXA.chord_5 & \\AXJ.chord_5', '\\AXA.chord_6 & \\AXJ.chord_6', '\\AXA.chord_7 & \\AXJ.chord_7', '\\AXA.chord_8 & \\AXJ.chord_8', '\\AXA.chord_9 & \\AXJ.chord_9', '\\AXA.chord_10 & \\AXJ.chord_10', '\\AXA.chord_11 & \\AXJ.chord_11', '\\AXA.chord_12 & \\AXJ.chord_12', '\\AXA.chord_13 & \\AXJ.chord_13', '\\AXA.chord_14 & \\AXJ.chord_14', '\\AXA.chord_15 & \\AXJ.chord_15', '\\AXA.chord_16 & \\AXJ.chord_16', '\\AXA.chord_17 & \\AXJ.chord_17', '\\AXA.chord_18 & \\AXJ.chord_18', '\\AXA.chord_19 & \\AXJ.chord_19', '\\AXA.chord_20 & \\AXJ.chord_20', '\\AXA.chord_21 & \\AXJ.chord_21', '\\AXA.chord_22 & \\AXJ.chord_22','\\xtomo::top.brightnesses.array_1:chord_01 & \\xtomo::top.brightnesses.array_3:chord_01', '\\xtomo::top.brightnesses.array_1:chord_02 & \\xtomo::top.brightnesses.array_3:chord_02', '\\xtomo::top.brightnesses.array_1:chord_03 & \\xtomo::top.brightnesses.array_3:chord_03', '\\xtomo::top.brightnesses.array_1:chord_04 & \\xtomo::top.brightnesses.array_3:chord_04', '\\xtomo::top.brightnesses.array_1:chord_05 & \\xtomo::top.brightnesses.array_3:chord_05', '\\xtomo::top.brightnesses.array_1:chord_06 & \\xtomo::top.brightnesses.array_3:chord_06', '\\xtomo::top.brightnesses.array_1:chord_07 & \\xtomo::top.brightnesses.array_3:chord_07', '\\xtomo::top.brightnesses.array_1:chord_08 & \\xtomo::top.brightnesses.array_3:chord_08', '\\xtomo::top.brightnesses.array_1:chord_09 & \\xtomo::top.brightnesses.array_3:chord_09', '\\xtomo::top.brightnesses.array_1:chord_10 & \\xtomo::top.brightnesses.array_3:chord_10', '\\xtomo::top.brightnesses.array_1:chord_11 & \\xtomo::top.brightnesses.array_3:chord_11', '\\xtomo::top.brightnesses.array_1:chord_12 & \\xtomo::top.brightnesses.array_3:chord_12', '\\xtomo::top.brightnesses.array_1:chord_13 & \\xtomo::top.brightnesses.array_3:chord_13', '\\xtomo::top.brightnesses.array_1:chord_14 & \\xtomo::top.brightnesses.array_3:chord_14', '\\xtomo::top.brightnesses.array_1:chord_15 & \\xtomo::top.brightnesses.array_3:chord_15', '\\xtomo::top.brightnesses.array_1:chord_16 & \\xtomo::top.brightnesses.array_3:chord_16', '\\xtomo::top.brightnesses.array_1:chord_17 & \\xtomo::top.brightnesses.array_3:chord_17', '\\xtomo::top.brightnesses.array_1:chord_18 & \\xtomo::top.brightnesses.array_3:chord_18', '\\xtomo::top.brightnesses.array_1:chord_19 & \\xtomo::top.brightnesses.array_3:chord_19', '\\xtomo::top.brightnesses.array_1:chord_20 & \\xtomo::top.brightnesses.array_3:chord_20', '\\xtomo::top.brightnesses.array_1:chord_21 & \\xtomo::top.brightnesses.array_3:chord_21', '\\xtomo::top.brightnesses.array_1:chord_22 & \\xtomo::top.brightnesses.array_3:chord_22', '\\xtomo::top.brightnesses.array_1:chord_23 & \\xtomo::top.brightnesses.array_3:chord_23', '\\xtomo::top.brightnesses.array_1:chord_24 & \\xtomo::top.brightnesses.array_3:chord_24', '\\xtomo::top.brightnesses.array_1:chord_25 & \\xtomo::top.brightnesses.array_3:chord_25','\\xtomo::top.brightnesses.array_1:chord_26 & \\xtomo::top.brightnesses.array_3:chord_26', '\\xtomo::top.brightnesses.array_1:chord_27 & \\xtomo::top.brightnesses.array_3:chord_27', '\\xtomo::top.brightnesses.array_1:chord_28 & \\xtomo::top.brightnesses.array_3:chord_28', '\\xtomo::top.brightnesses.array_1:chord_29 & \\xtomo::top.brightnesses.array_3:chord_29', '\\xtomo::top.brightnesses.array_1:chord_30 & \\xtomo::top.brightnesses.array_3:chord_30', '\\xtomo::top.brightnesses.array_1:chord_31 & \\xtomo::top.brightnesses.array_3:chord_31', '\\xtomo::top.brightnesses.array_1:chord_32 & \\xtomo::top.brightnesses.array_3:chord_32', '\\xtomo::top.brightnesses.array_1:chord_33 & \\xtomo::top.brightnesses.array_3:chord_33', '\\xtomo::top.brightnesses.array_1:chord_34 & \\xtomo::top.brightnesses.array_3:chord_34', '\\xtomo::top.brightnesses.array_1:chord_35 & \\xtomo::top.brightnesses.array_3:chord_35', '\\xtomo::top.brightnesses.array_1:chord_36 & \\xtomo::top.brightnesses.array_3:chord_36', '\\xtomo::top.brightnesses.array_1:chord_37 & \\xtomo::top.brightnesses.array_3:chord_37', '\\xtomo::top.brightnesses.array_1:chord_38 & \\xtomo::top.brightnesses.array_3:chord_38', '\\MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP01_ABK','\\MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP02_ABK','\\MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP02_GHK','\\MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP03_ABK','\\MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP04_ABK','\\MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP04_GHK','\\MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP05_ABK','\\MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP05_GHK','\\MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP06_ABK','\\MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP06_GHK','\\MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP07_ABK','\\MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP07_GHK','\\MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP08_GHK','\\MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP10_GHK','\\MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP11_GHK','\\MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP12_GHK','\\MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP13_GHK','\\MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP14_GHK','\\MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP08_ABK','\\MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP09_ABK','\\MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP1T_GHK','\\MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP20_ABK','\\MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP2T_GHK', '\\MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP3T_GHK', '\\MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP4T_ABK','\\MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP5T_ABK', '\\MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP6T_ABK','\\MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP_AB_TOP', '\\MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP_BC_BOT', '\\MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP_BC_TOP', '\\MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP_EF_BOT','Greenwald_fraction','n1_norm','\\efit_aeqdsk:q95','ip_err_frac',  'radiated_frac', 'rotating_mode' , '\\MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP_KA_TOP','\\MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP_EF_TOP', '\\MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP_KA_BOT','\\efit_aeqdsk:aminor', '\\efit_aeqdsk:betan', '\\efit_aeqdsk:kappa', '\\efit_aeqdsk:li', '\\efit_aeqdsk:wplasm']
shot_ids = [i for i in test_x_cmod_df['Shot_list'].values]

print(len(Diagnostics) , len(cmod_feats), train_x_cmod_df.shape , test_x_cmod_df.shape)
c = "\\"
for i in range (22):
    test_x_cmod_df[f'\AXA.chord_{i+1} & \AXJ.chord_{i+1}'] = test_x_cmod_df[f'\AXA.chord_{i+1}']/ test_x_cmod_df[f'\AXJ.chord_{i+1}']
    train_x_cmod_df[f'\AXA.chord_{i+1} & \AXJ.chord_{i+1}'] = train_x_cmod_df[f'\AXA.chord_{i+1}']/ train_x_cmod_df[f'\AXJ.chord_{i+1}']

for i in range (1 , 39):
    if i<10:
        test_x_cmod_df[f"{c}xtomo::top.brightnesses.array_1:chord_0{i} & {c}xtomo::top.brightnesses.array_3:chord_0{i}"] =  test_x_cmod_df[f'{c}xtomo::top.brightnesses.array_1:chord_0{i}']/ test_x_cmod_df[f'{c}xtomo::top.brightnesses.array_3:chord_0{i}']
        train_x_cmod_df[f'{c}xtomo::top.brightnesses.array_1:chord_0{i} & {c}xtomo::top.brightnesses.array_3:chord_0{i}'] = train_x_cmod_df[f'{c}xtomo::top.brightnesses.array_1:chord_0{i}']/ train_x_cmod_df[f'{c}xtomo::top.brightnesses.array_3:chord_0{i}']
    else:
        test_x_cmod_df[f'{c}xtomo::top.brightnesses.array_1:chord_{i} & {c}xtomo::top.brightnesses.array_3:chord_{i}'] = test_x_cmod_df[f'{c}xtomo::top.brightnesses.array_1:chord_{i}']/ test_x_cmod_df[f'{c}xtomo::top.brightnesses.array_3:chord_{i}']
        train_x_cmod_df[f'{c}xtomo::top.brightnesses.array_1:chord_{i} & {c}xtomo::top.brightnesses.array_3:chord_{i}'] = train_x_cmod_df[f'{c}xtomo::top.brightnesses.array_1:chord_{i}']/ train_x_cmod_df[f'{c}xtomo::top.brightnesses.array_3:chord_{i}']

train_x_cmod_df[['\\MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP01_ABK', '\\MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP02_ABK', '\\MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP03_ABK', '\\MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP04_ABK', '\\MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP05_ABK', '\\MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP06_ABK', '\\MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP09_ABK']]= np.NaN


# # rename jtext and hl_2a data features based on their equivalent c-mod selected minimal features 

train_x_cmod_df = train_x_cmod_df[cmod_feats].rename(columns = dict(zip(cmod_feats , Diagnostics)) )

test_x_cmod_df = test_x_cmod_df[cmod_feats].rename(columns = dict(zip(cmod_feats , Diagnostics)) )

train_y_cmod_df  = train_y_cmod_df.rename(columns={0:'Is_disrupt'})
train_y_cmod_df = train_y_cmod_df.reset_index().drop('index' , axis=1)

train_df_ = train_x_cmod_df
train_df_['Is_disrupt'] = train_y_cmod_df['Is_disrupt'].astype(int)

test_df_ = test_x_cmod_df
test_df_['Shot_list'] = shot_ids 

train_df_.to_csv(f'{files_path}/{CFG.Vc}_processed_sum_feature_extraction_train.csv' , index=False)  #2404 observations
test_df_.to_csv(f'{files_path}/{CFG.Vc}_processed_sum_feature_extraction_test.csv' , index=True)   #413 observations

112 112 (20, 173) (413, 198)


JTEXT RENAMING

In [17]:
Diagnostics = "plasma current,toroidal magnetic field,horizontal displacement,vertical displacement,line integral density(center chord),C3 radiation,loop voltage,AXUV_01,AXUV_02,AXUV_03,AXUV_04,AXUV_05,AXUV_06,AXUV_07,AXUV_08,AXUV_09,AXUV_10,AXUV_11,AXUV_12,AXUV_13,AXUV_14,AXUV_15,AXUV_16,AXUV_17,AXUV_18,AXUV_19,AXUV_20,AXUV_21,AXUV_22,soft-X-ray_01,soft-X-ray_02,soft-X-ray_03,soft-X-ray_04,soft-X-ray_05,soft-X-ray_06,soft-X-ray_07,soft-X-ray_08,soft-X-ray_09,soft-X-ray_10,soft-X-ray_11,soft-X-ray_12,soft-X-ray_13,soft-X-ray_14,soft-X-ray_15,soft-X-ray_16,soft-X-ray_17,soft-X-ray_18,soft-X-ray_19,soft-X-ray_20,soft-X-ray_21,soft-X-ray_22,soft-X-ray_23,soft-X-ray_24,soft-X-ray_25,soft-X-ray_26,soft-X-ray_27,soft-X-ray_28,soft-X-ray_29,soft-X-ray_30,soft-X-ray_31,soft-X-ray_32,soft-X-ray_33,soft-X-ray_34,soft-X-ray_35,soft-X-ray_36,soft-X-ray_37,soft-X-ray_38,poloidal Mirnov probes_01,poloidal Mirnov probes_02,poloidal Mirnov probes_03,poloidal Mirnov probes_04,poloidal Mirnov probes_05,poloidal Mirnov probes_06,poloidal Mirnov probes_07,poloidal Mirnov probes_08,poloidal Mirnov probes_09,poloidal Mirnov probes_10,poloidal Mirnov probes_11,poloidal Mirnov probes_12,toroidal Mirnov probes_01,toroidal Mirnov probes_02,Greenwald fraction,LM proxy,q95 proxy,IP error fraction,Radiation fraction,Rotating mode proxy".replace(' ','_').replace('-', '_').lower().split(',')
jtext_feats = "ip,bt,dx,dy,polaris_den_v09,vs_c3_aa018,vl,AXUV_CA_02,AXUV_CA_06,AXUV_CA_10,AXUV_CA_14,AXUV_CB_18,AXUV_CB_22,AXUV_CB_26,AXUV_CB_30,AXUV_CE_66,AXUV_CE_70,AXUV_CE_74,AXUV_CE_78,AXUV_CF_82,AXUV_CF_86,AXUV_CF_90,AXUV_CF_94,AXUV_CF_95,AXUV_CF_96,AXUV_CF_97,AXUV_CF_98,AXUV_CF_99,AXUV_CF_100,sxr_cb_020,sxr_cb_021,sxr_cb_022,sxr_cb_023,sxr_cb_024,sxr_cb_025,sxr_cb_026,sxr_cb_027,sxr_cb_028,sxr_cb_032,sxr_cb_036,sxr_cb_037,sxr_cb_038,sxr_cb_039,sxr_cb_040,sxr_cb_041,sxr_cb_042,sxr_cb_043,sxr_cb_044,sxr_cc_036,sxr_cc_037,sxr_cc_038,sxr_cc_039,sxr_cc_040,sxr_cc_041,sxr_cc_042,sxr_cc_043,sxr_cc_044,sxr_cc_052,sxr_cc_053,sxr_cc_054,sxr_cc_055,sxr_cc_056,sxr_cc_057,sxr_cc_058,sxr_cc_059,sxr_cc_060,sxr_cc_061,MA_POL_CA01T,MA_POL_CA03T,MA_POL_CA05T,MA_POL_CA07T,MA_POL_CA09T,MA_POL_CA11T,MA_POL_CA13T,MA_POL_CA15T,MA_POL_CA17T,MA_POL_CA19T,MA_POL_CA21T,MA_POL_CA23T,MA_TOR1_R01,MA_TOR1_R02,ne_nG,n = 1 amplitude,qa_proxy,ip_error,radiation_proxy,rotating_mode_proxy".split(',')
train_x_jtext_df[['AXUV_CF_95', 'AXUV_CF_96', 'AXUV_CF_97', 'AXUV_CF_98', 'AXUV_CF_99', 'AXUV_CF_100', 'sxr_cc_061', 'n = 1 amplitude']]=0
train_x_jtext_df = train_x_jtext_df[jtext_feats].rename(columns = dict(zip(jtext_feats , Diagnostics)) )

train_y_jtext_df  = train_y_jtext_df.rename(columns={0:'Is_disrupt'})
train_y_jtext_df = train_y_jtext_df.reset_index().drop('index' , axis=1)

train_df_ = train_x_jtext_df
train_df_['Is_disrupt'] = train_y_jtext_df['Is_disrupt'].astype(int)

train_df_.to_csv(f'{files_path}/{CFG.Vj}_processed_sum_feature_extraction_train.csv' , index=False)  #2404 observations


HL_2A RENAMING

In [18]:
Diagnostics = "plasma current,toroidal magnetic field,horizontal displacement,vertical displacement,line integral density(center chord),C3 radiation,loop voltage,AXUV_01,AXUV_02,AXUV_03,AXUV_04,AXUV_05,AXUV_06,AXUV_07,AXUV_08,AXUV_09,AXUV_10,AXUV_11,AXUV_12,AXUV_13,AXUV_14,AXUV_15,AXUV_16,soft-X-ray_01,soft-X-ray_02,soft-X-ray_03,soft-X-ray_04,soft-X-ray_05,soft-X-ray_06,soft-X-ray_07,soft-X-ray_08,soft-X-ray_09,soft-X-ray_10,soft-X-ray_11,soft-X-ray_12,soft-X-ray_13,soft-X-ray_14,soft-X-ray_15,soft-X-ray_16,soft-X-ray_17,soft-X-ray_18,soft-X-ray_19,soft-X-ray_20,poloidal Mirnov probes_01,poloidal Mirnov probes_02,poloidal Mirnov probes_03,poloidal Mirnov probes_04,poloidal Mirnov probes_05,poloidal Mirnov probes_06,poloidal Mirnov probes_07,poloidal Mirnov probes_08,poloidal Mirnov probes_09,poloidal Mirnov probes_10,poloidal Mirnov probes_11,poloidal Mirnov probes_12,poloidal Mirnov probes_13,poloidal Mirnov probes_14,poloidal Mirnov probes_15,poloidal Mirnov probes_16,poloidal Mirnov probes_17,poloidal Mirnov probes_18,toroidal Mirnov probes_01,toroidal Mirnov probes_02,toroidal Mirnov probes_03,toroidal Mirnov probes_04,toroidal Mirnov probes_05,toroidal Mirnov probes_06,toroidal Mirnov probes_07,toroidal Mirnov probes_09,toroidal Mirnov probes_10,Density_Measurement1,Density_Measurement2,Density_Measurement3,Density_Measurement4,Greenwald fraction,LM proxy,q95 proxy,IP error fraction,Radiation fraction,Rotating mode proxy,unknown_LFBBOH,unknown_PUFFCTRL,unknown_LFBBV,unknown_LFBIRF,unknown_LFBMP1,unknown_LFBMP2".replace(' ','_').replace('-', '_').lower().split(',')
hl_2a_feats = "CCO-LFB:LFEX-IP,CCO-LFB:LFBBT,CCO-LFB:LFDH,CCO-LFB:LFDV,CCO-DF:DENSITY1,CCO-DF:GASFBOUT,DS-EMD-ROG:VL-FILTER,DS-BM-AB:BOLD01,DS-BM-AB:BOLD02,DS-BM-AB:BOLD03,DS-BM-AB:BOLD04,DS-BM-AB:BOLD05,DS-BM-AB:BOLD06,DS-BM-AB:BOLD07,DS-BM-AB:BOLD08,DS-BM-AB:BOLD09,DS-BM-AB:BOLD10,DS-BM-AB:BOLD11,DS-BM-AB:BOLD12,DS-BM-AB:BOLD13,DS-BM-AB:BOLD14,DS-BM-AB:BOLD15,DS-BM-AB:BOLD16,DS-SXR-SXA:SX01,DS-SXR-SXA:SX02,DS-SXR-SXA:SX03,DS-SXR-SXA:SX04,DS-SXR-SXA:SX05,DS-SXR-SXA:SX06,DS-SXR-SXA:SX07,DS-SXR-SXA:SX08,DS-SXR-SXA:SX09,DS-SXR-SXA:SX10,DS-SXR-SXA:SX11,DS-SXR-SXA:SX12,DS-SXR-SXA:SX13,DS-SXR-SXA:SX14,DS-SXR-SXA:SX15,DS-SXR-SXA:SX16,DS-SXR-SXA:SX17,DS-SXR-SXA:SX18,DS-SXR-SXA:SX19,DS-SXR-SXA:SX20,DS-EMD-MP:MPOL-01,DS-EMD-MP:MPOL-02,DS-EMD-MP:MPOL-03,DS-EMD-MP:MPOL-04,DS-EMD-MP:MPOL-05,DS-EMD-MP:MPOL-06,DS-EMD-MP:MPOL-07,DS-EMD-MP:MPOL-08,DS-EMD-MP:MPOL-09,DS-EMD-MP:MPOL-10,DS-EMD-MP:MPOL-11,DS-EMD-MP:MPOL-12,DS-EMD-MP:MPOL-13,DS-EMD-MP:MPOL-14,DS-EMD-MP:MPOL-15,DS-EMD-MP:MPOL-16,DS-EMD-MP:MPOL-17,DS-EMD-MP:MPOL-18,DS-EMD-MP:NPOL-01,DS-EMD-MP:NPOL-02,DS-EMD-MP:NPOL-03,DS-EMD-MP:NPOL-04,DS-EMD-MP:NPOL-05,DS-EMD-MP:NPOL-06,DS-EMD-MP:NPOL-07,DS-EMD-MP:NPOL-09,DS-EMD-MP:NPOL-10,DS-FIR:FIR-SPH1A,DS-FIR:FIR-SPH2A,DS-FIR:FIR-SPH3A,DS-FIR:FIR-SPH4A,GW_Ratio,n1_norm,EFIT_q95,Ip_Error,RadiatedFraction,RotatingModeProxy,CCO-LFB:LFBBOH,DS-TMP:PUFFCTRL,CCO-LFB:LFBBV,CCO-LFB:LFBIRF,CCO-LFB:LFBMP1,CCO-LFB:LFBMP2".split(",")

train_x_hl_2a_df['n1_norm'] = 0
train_x_hl_2a_df = train_x_hl_2a_df[hl_2a_feats].rename(columns = dict(zip(hl_2a_feats , Diagnostics)) )

train_y_hl_2a_df  = train_y_hl_2a_df.rename(columns={0:'Is_disrupt'})
train_y_hl_2a_df = train_y_hl_2a_df.reset_index().drop('index' , axis=1)

train_df_ = train_x_hl_2a_df
train_df_['Is_disrupt'] = train_y_hl_2a_df['Is_disrupt'].astype(int)

train_df_.to_csv(f'{files_path}/{CFG.Vh}_processed_sum_feature_extraction_train.csv' , index=False)  #2404 observations


### Modelling

Blend of xgboost and Catboost with equal ratios

In [19]:
import xgboost as xgb
import catboost as catb


import sklearn.preprocessing as preprocess
import sklearn.model_selection as model_selection
import sklearn.metrics as metrics
from tqdm.auto import tqdm
import pandas as pd
import joblib


In [20]:
train_cmod_df = pd.read_csv(f'{files_path}/{CFG.Vc}_processed_sum_feature_extraction_train.csv')
train_jtext_df = pd.read_csv(f'{files_path}/{CFG.Vj}_processed_sum_feature_extraction_train.csv')
train_hl_2a_df = pd.read_csv(f'{files_path}/{CFG.Vh}_processed_sum_feature_extraction_train.csv')

test_cmod_df = pd.read_csv(f'{files_path}/{CFG.Vc}_processed_sum_feature_extraction_test.csv')

In [21]:
train_df = pd.concat([train_cmod_df , train_jtext_df , train_hl_2a_df])

In [22]:
train_df['programmed_plasma_current'] = train_df['plasma_current'] - train_df['ip_error_fraction']
test_cmod_df['programmed_plasma_current'] = test_cmod_df['plasma_current'] - test_cmod_df['ip_error_fraction']

The following features diagnosis were not known and no information about existed within the ITU provided variable sheet but since they were also part of tags in the test cmod, they were selected for training and given the identifier "unknown"

- unknown_bp_ka_top', 'unknown_bp_ef_top', 'unknown_bp_ka_bot', 'unknown_aminor', 'unknown_betan','unknown_kappa', 'unknown_li', 'unknown_wplasm'

 
The following features diagnosis were not known and no information about existed within the ITU provided variable sheet and since they werent also part of tags in the test cmod, they were given the identifier "unknown" and were not selected for training.

- 'unknown_lfbboh', 'unknown_puffctrl', 'unknown_lfbbv', 'unknown_lfbirf', 'unknown_lfbmp1'

it should be noted that if the above is placed under their correct diagnosis which were part of the selected features, they would be a major improvemnt in the model performance

In [23]:
feat_cols = ['plasma_current', 'toroidal_magnetic_field', 'horizontal_displacement', 'vertical_displacement', 'line_integral_density(center_chord)', 'c3_radiation',
 'loop_voltage', 'axuv_01', 'axuv_02', 'axuv_03', 'axuv_04', 'axuv_05', 'axuv_06', 'axuv_07', 'axuv_08', 'axuv_09', 'axuv_10', 'axuv_11', 'axuv_12', 'axuv_13',
 'axuv_14', 'axuv_15', 'axuv_16', 'axuv_17', 'axuv_18', 'axuv_19', 'axuv_20', 'axuv_21', 'axuv_22', 'soft_x_ray_01',
 'soft_x_ray_02', 'soft_x_ray_03', 'soft_x_ray_04', 'soft_x_ray_05', 'soft_x_ray_06', 'soft_x_ray_07', 'soft_x_ray_08', 'soft_x_ray_09',
 'soft_x_ray_10', 'soft_x_ray_11', 'soft_x_ray_12', 'soft_x_ray_13', 'soft_x_ray_14', 'soft_x_ray_15', 'soft_x_ray_16', 'soft_x_ray_17',
 'soft_x_ray_18', 'soft_x_ray_19', 'soft_x_ray_20', 'soft_x_ray_21', 'soft_x_ray_22', 'soft_x_ray_23', 'soft_x_ray_24', 'soft_x_ray_25',
 'soft_x_ray_26', 'soft_x_ray_27', 'soft_x_ray_28', 'soft_x_ray_29', 'soft_x_ray_30', 'soft_x_ray_31', 'soft_x_ray_32', 'soft_x_ray_33',
 'soft_x_ray_34', 'soft_x_ray_35', 'soft_x_ray_36', 'soft_x_ray_37', 'soft_x_ray_38', 'poloidal_mirnov_probes_01', 'poloidal_mirnov_probes_02',
 'poloidal_mirnov_probes_03', 'poloidal_mirnov_probes_04', 'poloidal_mirnov_probes_05', 'poloidal_mirnov_probes_06',
 'poloidal_mirnov_probes_07', 'poloidal_mirnov_probes_08', 'poloidal_mirnov_probes_09', 'poloidal_mirnov_probes_10',
 'poloidal_mirnov_probes_11', 'poloidal_mirnov_probes_12', 'poloidal_mirnov_probes_13', 'poloidal_mirnov_probes_14',
 'poloidal_mirnov_probes_15', 'poloidal_mirnov_probes_16', 'poloidal_mirnov_probes_17', 'poloidal_mirnov_probes_18',
 'toroidal_mirnov_probes_01', 'toroidal_mirnov_probes_02', 'toroidal_mirnov_probes_03', 'toroidal_mirnov_probes_04',
 'toroidal_mirnov_probes_05', 'toroidal_mirnov_probes_06', 'toroidal_mirnov_probes_07', 'toroidal_mirnov_probes_09',
 'toroidal_mirnov_probes_10', 'density_measurement1', 'density_measurement2', 'density_measurement3', 'density_measurement4', 'greenwald_fraction',
 'lm_proxy', 'q95_proxy', 'ip_error_fraction', 'radiation_fraction', 'rotating_mode_proxy', 'unknown_bp_ka_top', 'unknown_bp_ef_top', 'unknown_bp_ka_bot', 'unknown_aminor', 'unknown_betan', 'unknown_kappa', 'unknown_li', 'unknown_wplasm',
 'programmed_plasma_current']

In [24]:
def PIPELINE1(ytrain_label, fold_no=20, save_model=False):
    seed = 2021 # seed
    skf = model_selection.StratifiedKFold(n_splits=fold_no, shuffle=True, random_state=seed) # for cross validation
    scores = { 'F1_SCORE':[], 'LOG_SCORE':[], 'recall':[],'ACC':[]}
    preds= []

    ratio1 = 0.5
    ratio2 = 0.5

    X_ , y_ = train_df[feat_cols].fillna(np.inf).replace(np.NAN , np.inf) , train_df['Is_disrupt'].astype(int)
    test_new = test_cmod_df[feat_cols].fillna(np.inf).replace(np.NAN , np.inf)

    #creating a for loop for the kfold
    i = 1
    for train, test_ in skf.split(X_, y_ ):
        x_train, x_test, y_train, y_test = X_.iloc[train], X_.iloc[test_], y_.iloc[train], y_.iloc[test_]# spliting the data

        global estimator1 , estimator2
        
        estimator1 =  catb.CatBoostClassifier(iterations=40000,  has_time=True ,bootstrap_type='No',random_strength=0,
                                   learning_rate=0.011,
                                   use_best_model=True,
                                   reg_lambda=6,random_seed=39838,        
                                   eval_metric='F1',
                                   early_stopping_rounds=1000, verbose=True )
                                      
        estimator1.fit(x_train, y_train , eval_set=[(x_test, y_test)]  , verbose= 5000 ) 


        estimator2 =   xgb.XGBClassifier(learning_rate=0.0023,n_estimators=1501,random_state=42,missing = np.inf,
                                    scale_pos_weight=1.2, max_depth=40, reg_lambda=0.8) 

        estimator2.fit(x_train, y_train  ,eval_set=[(x_test, y_test)]  , verbose= 5000 )


            # save model
        if save_model:
            joblib.dump(estimator1, f'{models_path}/catb{i}.pkl')
            joblib.dump(estimator2, f'{models_path}/xgb{i}.pkl')

            # load model
            estimator1 = joblib.load(f'{models_path}/catb{i}.pkl')
            estimator2 = joblib.load(f'{models_path}/xgb{i}.pkl')

        pred1 = estimator1.predict_proba(test_new)[:, 1] # making predictions for test
        pred2 = estimator2.predict_proba(test_new)[:, 1] # making predictions for test
        
        print('Number of splits trained {} '.format(i))
        pred = pred1*ratio1 +pred2*ratio2 

        valid_probs = estimator1.predict_proba(x_test)[:, 1]*ratio1 + estimator2.predict_proba(x_test)[:, 1]*ratio2 #+ estimator3.predict_proba(x_test)[:, 1]*ratio3
        valid_preds = [0 if i<0.51 else 1 for i in valid_probs]

        f1_score_ = metrics.f1_score(y_test, valid_preds ,average='weighted')
        log_score = metrics.log_loss(y_test, valid_probs)
        recallscore = metrics.recall_score(y_test, valid_preds) # checking the accuracy
        ACCscore = metrics.accuracy_score(y_test, valid_preds) # checking the accuracy

        print(f'log_loss: {log_score} F1_score: {f1_score_} acc_score: {ACCscore} , recall: {recallscore}')

        scores['LOG_SCORE'].append(log_score)
        scores['F1_SCORE'].append(f1_score_)
        scores['recall'].append(recallscore)
        scores['ACC'].append(ACCscore)

        preds.append(pred)
        i+=1

    # fro the final score since predict was used, find mean over all folds used to group the predictions
    print(f" F1_SCORE: { np.mean(scores['F1_SCORE']) } | LOGLOSS_SCORE: { np.mean(scores['LOG_SCORE']) }  Recall: {np.mean(scores['recall'])} , ACC: {np.mean(scores['ACC'])}" )
    preds_mean = np.mean(preds, axis=0)
    return preds_mean

# lb 
final_df = pd.DataFrame({'Shot_list':test_cmod_df.Shot_list.values} ) # 'Is_disrupt':submit_df.Is_disrupt.values

final_targs = 'Is_disrupt'

for i in tqdm([final_targs]):
    final_df[i] = PIPELINE1(i ,  fold_no=12, save_model=True)

final_df['preds'] = final_df['Is_disrupt'].values
final_df['Is_disrupt'] =  [0 if i<0.51 else 1 for i in final_df['preds'].values]

final_df[['Shot_list' , 'Is_disrupt']].to_csv(f'{files_path}/{CFG.V}_MMD_PIPELINE1_{CFG.sub_no}.csv', index=False)
print('----------------------Done--------------------')

  0%|          | 0/1 [00:00<?, ?it/s]

0:	learn: 0.8789323	test: 0.9072165	best: 0.9072165 (0)	total: 172ms	remaining: 1h 54m 58s
Stopped by overfitting detector  (1000 iterations wait)

bestTest = 0.9644670051
bestIteration = 674

Shrink model to first 675 iterations.
[0]	validation_0-logloss:0.69124
[1500]	validation_0-logloss:0.11926
Number of splits trained 1 
log_loss: 0.10790344813582815 F1_score: 0.9654827720852577 acc_score: 0.9655172413793104 , recall: 0.9494949494949495
0:	learn: 0.8835085	test: 0.8514851	best: 0.8514851 (0)	total: 15.1ms	remaining: 10m 4s
Stopped by overfitting detector  (1000 iterations wait)

bestTest = 0.9702970297
bestIteration = 1549

Shrink model to first 1550 iterations.
[0]	validation_0-logloss:0.69118
[1500]	validation_0-logloss:0.10661
Number of splits trained 2 
log_loss: 0.09905999776344732 F1_score: 0.9732521409096914 acc_score: 0.9731800766283525 , recall: 0.98
0:	learn: 0.8801464	test: 0.8923077	best: 0.8923077 (0)	total: 10.6ms	remaining: 7m 3s
Stopped by overfitting detector  (10

In [25]:
final_df['Is_disrupt'].value_counts()

0    288
1    125
Name: Is_disrupt, dtype: int64